<a href="https://colab.research.google.com/github/zmy6208/zmy6208/blob/main/Pytorch_Lightning_csimcse_classification_20240629.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 実行環境の設定

In [ ]:
###
# GPUおよびPython環境のチェック
#import locale
#locale.getpreferredencoding = lambda: "UTF-8"
!nvidia-smi
!python --version

###
# Google Drive のマウント
from google.colab import drive
drive.mount('/content/gdrive')

Sat Jun 29 07:19:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0              52W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# パッケージインポート

In [ ]:
###
import numpy as np
import pandas as pd
import json
import openpyxl as op
import time
import os
import sys

#
!pip install transformers
!pip install sentence_transformers
#!pip install torch
#from torch import torch
#from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

!pip install pytorch_lightning --quiet
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl

#
#from sklearn.metrics import adjusted_mutual_info_score
#from sklearn.metrics import confusion_matrix, classification_report
#from sklearn.metrics.pairwise import paired_cosine_distances
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
#
#from sklearn.decomposition import PCA
#from sklearn.preprocessing import minmax_scale

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.3/812.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 14.9 MB/s eta 0:00:00


# パラメータ設定

In [ ]:
### cyclone simcseのパス
cyclone_model_name_or_path = "/content/gdrive/MyDrive/cyclone-simcse-chinese-roberta-wwm-ext"

### excelデータファイルパス
PATH_SEPARATOR = "/"
#
INPUT_DIR = "/content/gdrive/MyDrive/csimcse_data/input/Chinese_testdata/"
OUTPUT_DIR = "/content/gdrive/MyDrive/csimcse_data/output/Chinese_testdata/"
#
INPUT_FILE = "/content/gdrive/MyDrive/csimcse_data/input/Corpus-Review+Corpus-Novel.xlsx"

### 11感情カテゴリ
EMOTION_CATEGORIES = ['joy','anger','sadness','fear','shame','like','dislike','excitement','peacefulness','surprise','request']

### corpusで実行(True)、またはirisテストで実行(False)
CORPUS_MODE = True

### DNNパラメータ
AVAIL_GPUS = min(1, torch.cuda.device_count())
#print(AVAIL_GPUS)
DROPOUT = 0.5
VERBOSE = True
BATCH_SIZE = 10
EPOCK = 100

###
MODEL_SAVE_PATH = "/content/gdrive/MyDrive/csimcse_data/model/"

# ファイル入出力ユーティリティ

In [ ]:
### jsonファイル入出力
# jsonファイルの読み込み：必要な列数に書き換える
def read_json_file(JSON_DATAFILE):
    pd_data = pd.read_json(JSON_DATAFILE)
    ndaray_data = pd_data.to_numpy(copy=True).transpose()
    list_first_column = list(ndaray_data[0])
    list_second_column = list(ndaray_data[1])
    #print(list_first_column)
    #print(list_second_column)
    return list_first_column, list_second_column

# 2次元list_dataリストのjsonファイルへの書き出し
def write_json_file(JSON_DATAFILE, list_data):
    with open(JSON_DATAFILE, "w") as f:
        json.dump(list_data, f)

### excelファイル入出力
# excelデータの読み込み
def read_excel_file(EXCEL_DATAFILE):
    input_book = pd.ExcelFile(EXCEL_DATAFILE)
    input_sheet_name = input_book.sheet_names
    input_sheet_df = input_book.parse(input_sheet_name[0])
    transposed_df = input_sheet_df.transpose()
    data = transposed_df.values.tolist()
    #print("Number of columns:", len(data))
    if len(data) == 13:                              # Corpus for emotional analysis
        sentences = data[1]
        emotion_data = np.array(data[2:13])
    else:
        print("Excel : not supported format")
        sys.exit()
    return len(data), sentences, emotion_data

# excelファイルへの書き出し
def write_excel_file(EXCEL_DATAFILE, pd_data):
    pd_data.to_excel(EXCEL_DATAFILE, index=False, header=False)

## textファイルの入出力
# textデータの読み込み
def read_text_file(TEXT_DATAFILE):
    sentences = []
    with open(TEXT_DATAFILE, mode='r', encoding='utf-8') as f:
        for s in f:
            try:
                sentences.append(s.rstrip())
            except Exception as e:
                print(e)
    return sentences

# textファイルへの書き出し
def write_text_file(TEXT_DATAFILE, sentences):
    with open(TEXT_DATAFILE, mode='w', encoding='utf-8') as f:
        f.write('\n'.join(sentences))

# Chinese SimCSEによる文埋め込みの生成

In [ ]:
### Chinese SimCSEモデル
# Cyclone simcseのパス
cyclone_model_name_or_path = "/content/gdrive/MyDrive/cyclone-simcse-chinese-roberta-wwm-ext"
# Chinese SimCSEモデルの生成
cyclone_simcse_model = SentenceTransformer(cyclone_model_name_or_path)

### 文埋め込みと各感情情報の取得
def obtain_sentence_emdeddings_categories():
    print("Corpusの読込")
    emotion_labels = {}         # {感情カテゴリ, 全文に対する感情を持つ(1)か否か(0)を表すベクトル}
    _num, sentences, emotion_data = read_excel_file(INPUT_FILE)
    #print(num)
    #
    print("文埋め込みの生成開始")
    sentence_embeddings = cyclone_simcse_model.encode(sentences)
    #print(type(sentence_embeddings), sentence_embeddings.shape)
    #
    print("各感情ベクトルの生成")
    for e, label in enumerate(emotion_data):
        #print(EMOTION_CATEGORIES[e])
        #print(e, len(label))
        emotion_labels[EMOTION_CATEGORIES[e]] = label
    return sentences, sentence_embeddings, emotion_labels

def check_emotional_categories():
    sentences, sentence_embeddings, emotion_labels = obtain_sentence_emdeddings_categories()
    print(type(sentences), type(sentence_embeddings))
    print("Total senteces:", len(sentences))
    for k, v in emotion_labels.items():
        print(type(v))
        print(k, np.count_nonzero(v == 1))

#check_emotional_categories()

# Pytorch Lightningを用いたDNN

In [ ]:
###
def split_train_val_test(x, t):
    # PyTorch の Tensor 型へ変換
    x = torch.tensor(x, dtype=torch.float32)
    t = torch.tensor(t, dtype=torch.int64)

    # 入力値と目標値をまとめる
    dataset = torch.utils.data.TensorDataset(x, t)

    # 各データセットのサンプル数を決定
    # train : val: test = 60%　: 20% : 20%
    n_train = int(len(dataset) * 0.6)
    n_val = int(len(dataset) * 0.2)
    n_test = len(dataset) - n_train - n_val

    # データセットの分割
    # ランダムに分割を行うため、シードを固定して再現性を確保
    torch.manual_seed(0)
    train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])
    n_test = len(dataset) - n_train - n_val
    return train, val, test

###
# Chinese corpusのtrain, val, testデータの取得
def obtain_corpus_train_val_test(num):
    _sentences, sentence_embeddings, emotion_labels = obtain_sentence_emdeddings_categories()
    #print(sentence_embeddings.shape, len(sentence_embeddings[0]), emotion_labels[EMOTION_CATEGORIES[num]].shape)
    train, val, test = split_train_val_test(sentence_embeddings, emotion_labels[EMOTION_CATEGORIES[num]])
    total_size = len(sentence_embeddings)
    in_dim = len(sentence_embeddings[0])
    out_dim = len(np.unique(emotion_labels[EMOTION_CATEGORIES[num]]))
    #print(total_size,in_dim, out_dim)
    return total_size, in_dim, out_dim, train, val, test

#obtain_corpus_train_val_test()

# DNNモデルの定義

In [ ]:
###
def generate_DNN_model(in_dim, out_dim, train, val, test):
    # バッチサイズの定義
    batch_size = BATCH_SIZE

    # Data Loader を用意
    # shuffle はデフォルトで False のため、訓練データのみ True に指定
    train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
    val_loader = torch.utils.data.DataLoader(val, batch_size)
    test_loader = torch.utils.data.DataLoader(test, batch_size)

    class Network(pl.LightningModule):

        def __init__(self):
            super().__init__()
            input_dim=in_dim
            hidden_dim=in_dim
            output_dim=out_dim
            dropout=DROPOUT
            self.dropout = nn.Dropout(dropout)
            self.batch_size = batch_size
            self.fc1 = nn.Linear(in_dim, in_dim)
            self.fc2 = nn.Linear(in_dim, in_dim)
            self.fc3 = nn.Linear(in_dim, in_dim)
            #self.fc4 = nn.Linear(in_dim, in_dim)
            #self.fc5 = nn.Linear(in_dim, in_dim)
            #self.fc6 = nn.Linear(in_dim, in_dim)
            self.fc_out = nn.Linear(in_dim, out_dim)
            self.bn = nn.BatchNorm1d(in_dim)

        def forward(self, x):
            x = self.bn(x)
            h = self.fc1(x)
            h = F.relu(h)
            h = self.dropout(h)
            h = self.fc2(h)
            h = F.relu(h)
            h = self.dropout(h)
            h = self.fc3(h)
            h = F.relu(h)
            h = self.dropout(h)
            #h = self.fc4(h)
            #h = F.relu(h)
            #h = self.dropout(h)
            #h = self.fc5(h)
            #h = F.relu(h)
            #h = self.dropout(h)
            #h = self.fc6(h)
            #h = F.relu(h)
            #h = self.dropout(h)
            h = self.fc_out(h)
            return h

        def training_step(self, batch, batch_idx):
            x, t = batch
            y = self(x)
            loss = F.cross_entropy(y, t)
            y_label = torch.argmax(y, dim=1)
            acc = torch.sum(t == y_label) * 1.0 / len(t)
            self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
            self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True)
            return loss

        def validation_step(self, batch, batch_idx):
            x, t = batch
            y = self(x)
            loss = F.cross_entropy(y, t)
            y_label = torch.argmax(y, dim=1)
            acc = torch.sum(t == y_label) * 1.0 / len(t)
            self.log('val_loss', loss, on_step=False, on_epoch=True)
            self.log('val_acc', acc, on_step=False, on_epoch=True)
            return loss

        def test_step(self, batch, batch_idx):
            x, t = batch
            y = self(x)
            loss = F.cross_entropy(y, t)
            y_label = torch.argmax(y, dim=1)
            acc = torch.sum(t == y_label) * 1.0 / len(t)
            self.log('test_loss', loss, on_step=False, on_epoch=True)
            self.log('test_acc', acc, on_step=False, on_epoch=True)
            return loss

        def configure_optimizers(self):
            optimizer = torch.optim.SGD(self.parameters(), lr=0.01)
            #optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
            return optimizer

    # シードを固定して再現性を確保
    pl.seed_everything(0)

    ###
    # 学習を行う Trainer
    net = Network()
    #trainer = pl.Trainer(gpus=1, max_epochs=30, deterministic=True)
    #trainer = pl.Trainer(devices=1, max_epochs=30, deterministic=True)
    #trainer = pl.Trainer(accelerator='gpu', devices=1, max_epochs=30, deterministic=True)
    trainer = pl.Trainer(accelerator='auto', devices=AVAIL_GPUS, max_epochs=EPOCK, deterministic=True,
        callbacks=[
            # early stopping
            #pl.callbacks.EarlyStopping(monitor='val_acc', mode='max', patience=15, verbose=VERBOSE),
            pl.callbacks.EarlyStopping(monitor='val_acc', mode='max', patience=10, verbose=VERBOSE),
            # checkpoint,
            pl.callbacks.ModelCheckpoint(save_top_k=1, monitor="val_acc", mode="max",verbose=VERBOSE),
            #pl.callbacks.LearningRateMonitor(logging_interval='epoch',verbose=verbose),
            pl.callbacks.TQDMProgressBar(refresh_rate=0 if not VERBOSE else 10),
        ],
        enable_checkpointing=True,
        logger= False
    )
    return net, train_loader, val_loader, test_loader, trainer

# 分類の実行

In [ ]:
###
# DNNネット構造を作成
_total_size, in_dim, out_dim, train, val, test = obtain_corpus_train_val_test(0)
net, _train_loader, _val_loader, _test_loader, trainer = generate_DNN_model(in_dim, out_dim, train, val, test)

# 入力文埋め込みの生成
_num, sentences, emotion_data = read_excel_file(INPUT_FILE)
sentence_embeddings = cyclone_simcse_model.encode(sentences)
tensor_embeddings = torch.tensor(sentence_embeddings, dtype=torch.float32)

# 格納済みモデルを読み込み
precisions = []
recalls = []
f1s = []
for e, category in enumerate(EMOTION_CATEGORIES):
    print(category)
    category_vector = emotion_data[e]
    print(category + "がポジティブの教師文数：", np.count_nonzero(category_vector))
    net.load_state_dict(torch.load(MODEL_SAVE_PATH + category + ".pt"))
    #
    net.eval()
    #
    out = net(tensor_embeddings)
    out_argmax = torch.argmax(out, dim=1)
    #print(out_argmax.shape)
    #print(out_argmax)
    print(category + "がポジティブの予測文数：", torch.count_nonzero(out_argmax).numpy())
    #print(category_vector.shape, out_argmax.numpy().shape)
    #print(category_vector[:5], out_argmax.numpy()[:5])
    precision = precision_score(category_vector, out_argmax.numpy())
    recall = recall_score(category_vector, out_argmax.numpy())
    f1 = f1_score(category_vector, out_argmax.numpy())
    print("Precision:",precision,"Recall:",recall, "F1:",f1)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    print()

print("Average precision:", np.mean(precisions))
print("Average recall:", np.mean(recalls))
print("Average F1:", np.mean(f1s))

Corpusの読込
文埋め込みの生成開始


INFO:lightning_fabric.utilities.seed:Seed set to 0
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


各感情ベクトルの生成
joy
joyがポジティブの教師文数： 3127
joyがポジティブの予測文数： 3048
Precision: 0.9530839895013123 Recall: 0.9290054365206268 F1: 0.9408906882591094

anger
angerがポジティブの教師文数： 1186
angerがポジティブの予測文数： 910
Precision: 0.6505494505494506 Recall: 0.4991568296795953 F1: 0.564885496183206

sadness
sadnessがポジティブの教師文数： 477
sadnessがポジティブの予測文数： 231
Precision: 0.8095238095238095 Recall: 0.3920335429769392 F1: 0.5282485875706214

fear
fearがポジティブの教師文数： 192
fearがポジティブの予測文数： 106
Precision: 0.8301886792452831 Recall: 0.4583333333333333 F1: 0.5906040268456376

shame
shameがポジティブの教師文数： 40
shameがポジティブの予測文数： 0
Precision: 0.0 Recall: 0.0 F1: 0.0

like
likeがポジティブの教師文数： 1737


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


likeがポジティブの予測文数： 1849
Precision: 0.8561384532179557 Recall: 0.9113413932066782 F1: 0.8828778583379812

dislike
dislikeがポジティブの教師文数： 2972
dislikeがポジティブの予測文数： 2964
Precision: 0.868421052631579 Recall: 0.8660834454912517 F1: 0.8672506738544474

excitement
excitementがポジティブの教師文数： 113
excitementがポジティブの予測文数： 29
Precision: 0.9655172413793104 Recall: 0.24778761061946902 F1: 0.39436619718309857

peacefulness
peacefulnessがポジティブの教師文数： 207
peacefulnessがポジティブの予測文数： 107
Precision: 0.8504672897196262 Recall: 0.4396135265700483 F1: 0.5796178343949046

surprise
surpriseがポジティブの教師文数： 263
surpriseがポジティブの予測文数： 108
Precision: 0.8055555555555556 Recall: 0.33079847908745247 F1: 0.46900269541778977

request
requestがポジティブの教師文数： 556
requestがポジティブの予測文数： 488
Precision: 0.944672131147541 Recall: 0.829136690647482 F1: 0.8831417624521072

Average precision: 0.775828877497402
Average recall: 0.5366627534666251
Average F1: 0.609171438227173
